In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.4/704.4 kB 12.6 MB/s eta 0:00:00a 0:00:01


In [4]:
import torch
import torch.nn as nn
import json
import numpy as np
import pandas as pd
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import trimesh
import scipy.io
from torchvision.datasets import ImageFolder
import sys
sys.path.append('/kaggle/input/models/byop-2425/models')
from gan import Discriminator, Generator
# import sys
# sys.path.append("../models")
# from gan import Discriminator, Generator

In [6]:
mesh = trimesh.load('/kaggle/input/dataset/model/bed/IKEA_BEDDINGE/model.obj')
mesh_v = list(mesh.geometry.values())[0]
vertices = np.array(mesh_v.vertices)
faces = np.array(mesh_v.faces)
print(vertices)
print(faces)

[[-0.43324038 -0.07641411  0.10654314]
 [-0.43324038 -0.07641411  0.10654314]
 [-0.43324038 -0.07641411  0.10654314]
 [ 0.43324038 -0.07641411 -0.22230652]
 [ 0.43324038 -0.07641411  0.10654314]
 [ 0.43324038 -0.07641411  0.10654314]
 [ 0.43324038 -0.07641411  0.10654314]
 [-0.43324038 -0.07641411 -0.22230652]
 [-0.43324038 -0.07641411 -0.22230652]
 [-0.43324038 -0.07641411 -0.22230652]
 [-0.43324038 -0.0459173  -0.22230652]
 [-0.43324038 -0.0459173  -0.22230652]
 [-0.43324038 -0.0459173  -0.22230652]
 [-0.43324038 -0.04256963 -0.22186802]
 [-0.43324038 -0.04256963 -0.22186802]
 [-0.43324038 -0.04256963 -0.22186802]
 [-0.43324038 -0.039453   -0.2205761 ]
 [-0.43324038 -0.039453   -0.2205761 ]
 [-0.43324038 -0.039453   -0.2205761 ]
 [-0.43324038 -0.03677486 -0.21852035]
 [-0.43324038 -0.03677486 -0.21852035]
 [-0.43324038 -0.03677486 -0.21852035]
 [-0.43324038 -0.03471911 -0.21584221]
 [-0.43324038 -0.03471911 -0.21584221]
 [-0.43324038 -0.03471911 -0.21584221]
 [-0.43324038 -0.03342719

In [5]:
from torch.nn.parallel import DataParallel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
json_dir = "/kaggle/input/dataset/pix3d.json"
with open(json_dir, 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)

In [9]:
df

,img,category,img_size,2d_keypoints,mask,img_source,model,model_raw,model_source,3d_keypoints,voxel,rot_mat,trans_mat,focal_length,cam_position,inplane_rotation,truncated,occluded,slightly_occluded,bbox
0,img/bed/0001.png,bed,"[395, 244]","[[[182.5, 147.09375], [174.5, 225.09375], [16....",mask/bed/0001.png,ikea,model/bed/IKEA_MALM_2/model.obj,None,ikea,model/bed/IKEA_MALM_2/3d_keypoints.txt,model/bed/IKEA_MALM_2/voxel.mat,"[[0.7813941591465821, 0.00095539348511137, -0....","[-0.00024347016915001151, 0.09068297313399999,...",35.270398,"[-0.7062195301276326, 0.2367305448542897, -0.8...",-0.078517,False,False,False,"[4, 22, 362, 228]"
1,img/bed/0002.png,bed,"[1007, 599]","[[[309.29285714285714, 543.9148660714286], [-1...",mask/bed/0002.png,ikea,model/bed/IKEA_MALM_2/model.obj,None,ikea,model/bed/IKEA_MALM_2/3d_keypoints.txt,model/bed/IKEA_MALM_2/voxel.mat,"[[0.6331473196939317, 0.08400992130502279, -0....","[0.025652375712099995, 0.0434050556712, 1.1086...",32.378901,"[-0.8365679093356918, 0.3969797870961433, -0.6...",-0.107273,True,True,False,"[46, 47, 927, 599]"
2,img/bed/0003.png,bed,"[372, 292]","[[[308.0, 202.09375], [-1.0, -1.0], [-1.0, -1....",mask/bed/0003.png,ikea,model/bed/IKEA_HEMNES_1/model.obj,None,ikea,model/bed/IKEA_HEMNES_1/3d_keypoints.txt,model/bed/IKEA_HEMNES_1/voxel.mat,"[[0.9337851369875168, 0.004246139989357373, -0...","[0.05160487022685999, 0.0022441725076000067, 0...",39.511348,"[-0.33341418148967134, 0.18052455616467913, -0...",-0.019073,True,False,False,"[0, 33, 344, 292]"
3,img/bed/0004.png,bed,"[1063, 755]","[[[230.06357142857144, 399.5266517857143], [93...",mask/bed/0004.png,ikea,model/bed/IKEA_HEMNES_1/model.obj,None,ikea,model/bed/IKEA_HEMNES_1/3d_keypoints.txt,model/bed/IKEA_HEMNES_1/voxel.mat,"[[0.5029912563246631, 0.09073372306215703, -0....","[0.02463177822849999, 0.0738148517616, 1.09891...",31.871608,"[-0.9464253871842163, 0.2661593054097636, -0.4...",-0.120969,False,False,False,"[89, 74, 984, 738]"
4,img/bed/0005.png,bed,"[414, 449]","[[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [4...",mask/bed/0005.png,ikea,model/bed/IKEA_MALM_2/model.obj,None,ikea,model/bed/IKEA_MALM_2/3d_keypoints.txt,model/bed/IKEA_MALM_2/voxel.mat,"[[0.7883156484286317, -0.04024974826747893, 0....","[-0.205655543756, 0.057996630364799975, 1.6363...",89.131102,"[1.1526451872054488, 0.3727803200217579, -1.12...",-0.083036,True,False,True,"[38, 44, 414, 420]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10064,img/wardrobe/0239.jpg,wardrobe,"[500, 500]","[[[196.0, 37.09375], [268.0, 29.09375], [312.0...",mask/wardrobe/0239.png,internet,model/wardrobe/IKEA_PAX_3/model.obj,None,ikea,model/wardrobe/IKEA_PAX_3/3d_keypoints.txt,model/wardrobe/IKEA_PAX_3/voxel.mat,"[[0.8671943915986312, 0.008398518548578844, -0...","[0.00031610186885999925, -0.004190586852552, 1...",56.579773,"[-0.9892567666445963, 0.18039947720639815, -1....",0.008705,False,False,False,"[192, 25, 307, 476]"
10065,img/wardrobe/0240.jpg,wardrobe,"[500, 500]","[[[78.72729492, 56.09660339], [108.7272949, 54...",mask/wardrobe/0240.png,internet,model/wardrobe/IKEA_PAX_4/model.obj,None,ikea,model/wardrobe/IKEA_PAX_4/3d_keypoints.txt,model/wardrobe/IKEA_PAX_4/voxel.mat,"[[0.8537182493805817, 0.003437021039660772, -0...","[0.018082997613680003, 0.009325893335790002, 2...",69.363343,"[-1.069822307920201, 0.2115633698665138, -1.71...",-0.006371,False,False,False,"[67, 35, 427, 473]"
10066,img/wardrobe/0241.jpg,wardrobe,"[720, 960]","[[[235.2, 651.5571428571429], [491.65714285714...",mask/wardrobe/0241.png,ikea,model/wardrobe/IKEA_ANEBODA_1/model.obj,None,ikea,model/wardrobe/IKEA_ANEBODA_1/3d_keypoints.txt,model/wardrobe/IKEA_ANEBODA_1/voxel.mat,"[[0.9996163484519796, -0.005418352840159608, 0...","[0.008426788430139984, 0.12617876966199998, 2....",65.977675,"[0.0488976167628434, 0.4217906354965945, -2.02...",-0.061166,False,False,False,"[196, 58, 518, 716]"
10067,img/wardrobe/0242.jpg,wardrobe,"[500, 500]","[[[152.53125, 46.7265625], [3

In [7]:
class pix3d_dataset(Dataset):
    def __init__(self, dataframe, transform=None, data_dir='/kaggle/input/dataset/'):
        self.transform = transform
        self.dataframe = dataframe
        self.data_dir = data_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.data_dir + self.dataframe.iloc[idx]['img']
        mask_path = self.data_dir + self.dataframe.iloc[idx]['mask']
        voxel_path = self.data_dir + self.dataframe.iloc[idx]['voxel']

        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        voxel = scipy.io.loadmat(voxel_path)['voxel']

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        voxel = torch.tensor(voxel, dtype=torch.float32).unsqueeze(0)
        sample = {
            'image': image,
            'mask': mask,
            'voxel': voxel
        }
        return sample


In [8]:
latent_dim = 100
hidden_dim = 64
lr = 0.0002
batch_size = 1
num_epochs = 2

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
dataset = pix3d_dataset(df, transform = transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

generator = Generator(img_dim=3, hidden_dim=hidden_dim, latent_dim=latent_dim, output_dim=128).to(device)
# if torch.cuda.device_count() > 1:
#     generator = torch.nn.DataParallel(generator)
generator = generator.to('cuda:0')
discriminator = Discriminator(voxel_dim=1, img_dim=3, hidden_dim=hidden_dim).to(device)
# if torch.cuda.device_count() > 1:
#     discriminator = torch.nn.DataParallel(discriminator)
discriminator = discriminator.to("cuda:1")
optimizer_g = optim.SGD(generator.parameters(), lr=lr)
optimizer_d = optim.SGD(generator.parameters(), lr=lr)
criterion = nn.BCELoss()

In [9]:
for i, data in enumerate(dataloader):
    real_images = data['image'].to(device)
    real_voxels = data['voxel'].to(device)
    print("Image shape:", real_images.shape)
    print("Voxel shape:", real_voxels.shape)
    break

Image shape: torch.Size([1, 3, 128, 128])
Voxel shape: torch.Size([1, 1, 128, 128, 128])


In [10]:
import tqdm as tqdm

In [11]:
for epoch in tqdm.tqdm(range(num_epochs)):
    for i, data in enumerate(dataloader):
        real_images = data['image'].to(device)
        real_voxels = data['voxel'].to(device)

        print(f"Real images shape: {real_images.shape}")
        print(f"Real voxels shape: {real_voxels.shape}")

        batch_size = real_images.size(0)
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        #trainign Discriminator
        optimizer_d.zero_grad()

        #real data
        outputs = discriminator(real_voxels, real_images)
        d_loss_real = criterion(outputs, real_labels)

        #fake data
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_voxels = generator(real_images, z)

        print(f"Fake voxels shape: {fake_voxels.shape}")

        outputs = discriminator(fake_voxels.detach(), real_images)
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_d.step()

        optimizer_g.zero_grad()

        fake_voxels = generator(real_images, z)
        outputs = discriminator(fake_voxels, real_images)
        g_loss = criterion(outputs, real_labels)

        g_loss.backward()
        optimizer_g.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

    torch.save(generator.state_dict(), f'generator_epoch_{epoch+1}.pth')
    torch.save(discriminator.state_dict(), f'discriminator_epoch_{epoch+1}.pth')

  0%|          | 0/2 [00:00<?, ?it/s]


Real images shape: torch.Size([1, 3, 128, 128])
Real voxels shape: torch.Size([1, 1, 128, 128, 128])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)